1.Из датасета были извлечены изображения и аннотации класса 0  - 497

2.Утилитой yolo_bbox2segment BB были переведены в разметку полигонами ( ~4 минуты)

3.Преобразование координат полигонов в нормализованные координаты OBB (<1)
Добавлена проверка, если для OBB найдена координата больше 1, то в аннотацию заносятся координаты ограничивающего прямоугольника с углом поворота 0.

4.Подготовка датасета для обучения YOLOv8 OBB: разбивка на train и val, создание data.yaml

5.Обучение модели с весами "yolov8l-obb.pt" в течение 20 эпох.
Результат: MAP50 = 0.666. Время обучения 9 минут.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Укажите путь к архиву .zip на Google Диске
zip_path = '/content/drive/MyDrive/VMdata/DATASET_RAZMETKA-001.zip'

# Укажите путь для распаковки архива
extract_path = '/content/'

# Распаковка архива
!unzip "$zip_path" -d "$extract_path"

Archive:  /content/drive/MyDrive/VMdata/DATASET_RAZMETKA-001.zip
  inflating: /content/DATASET_RAZMETKA/labels/3Class_00242.txt  
  inflating: /content/DATASET_RAZMETKA/labels/3Class_00186.txt  
  inflating: /content/DATASET_RAZMETKA/labels/3Class_00194.txt  
  inflating: /content/DATASET_RAZMETKA/labels/3Class_00263.txt  
  inflating: /content/DATASET_RAZMETKA/labels/3Class_00211.txt  
  inflating: /content/DATASET_RAZMETKA/labels/3Class_00256.txt  
  inflating: /content/DATASET_RAZMETKA/labels/3Class_00181.txt  
  inflating: /content/DATASET_RAZMETKA/labels/3Class_00184.txt  
  inflating: /content/DATASET_RAZMETKA/labels/3Class_00232.txt  
  inflating: /content/DATASET_RAZMETKA/labels/3Class_00185.txt  
  inflating: /content/DATASET_RAZMETKA/labels/3Class_00244.txt  
  inflating: /content/DATASET_RAZMETKA/labels/3Class_00261.txt  
  inflating: /content/DATASET_RAZMETKA/labels/3Class_00197.txt  
  inflating: /content/DATASET_RAZMETKA/labels/3Class_00269.txt  
  inflating: /content/DAT

**Извлечение изображений и аннотаций класса 0 , запись их в /content/VMdataset**

In [ ]:

import os
import random
import shutil

def remove_file( file_path ):
  # Проверяем существование файла перед удалением
  if os.path.exists(file_path):
    os.remove(file_path)

# Пути к исходным директориям
source_images_dir = '/content/DATASET_RAZMETKA/images'
source_labels_dir = '/content/DATASET_RAZMETKA/labels'

# Пути к целевым директориям
images_dir = '/content/VMdataset/images'
labels_dir = '/content/VMdataset/labels'

# Создание целевых директорий, если они не существуют
os.makedirs(images_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)


# Для выбора файлов с названиями, начинающимися с '0Class_' в папке с изображениями
image_files = [f for f in os.listdir(source_images_dir) if f.startswith('0Class_') and f.endswith('.jpg')]
# Сортируем файлы в лексикографическом порядке
image_files.sort()

# Для выбора файлов с названиями, начинающимися с '0Class_' в папке с текстовыми файлами
label_files = [f for f in os.listdir(source_labels_dir) if f.startswith('0Class_') and f.endswith('.txt')]
# Сортируем файлы в лексикографическом порядке
label_files.sort()

# Переменная для нумерации файлов
count = 1

# Проходим по каждому файлу c изображением и копируем его в целевую директорию с новым именем
for ifile in image_files:
    # Форматируем номер файла для имени вида 00001.jpg, 00002.jpg и т.д.
    new_name = f"{count:05}.jpg"
    # Копируем файл из исходной директории в целевую с новым именем
    shutil.copy(os.path.join(source_images_dir, ifile), os.path.join(images_dir, new_name))
    count += 1

print("Количество изображений в списке image_files:", len(image_files))
print("Название последнего файла:", new_name)

count = 1
# Проходим по каждому файлу аннотаций и копируем его в целевую директорию с новым именем
for lfile in label_files:
    # Форматируем номер файла для имени вида 00001.txt, 00002.txt и т.д.
    new_name = f"{count:05}.txt"
    # Копируем файл из исходной директории в целевую с новым именем
    shutil.copy(os.path.join(source_labels_dir, lfile), os.path.join(labels_dir, new_name))
    count += 1

Количество изображений в списке image_files: 497
Название последнего файла: 00497.jpg


In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [ ]:
# Скрипт преобразования BB в контуры сегментации
from ultralytics.data.converter import yolo_bbox2segment

yolo_bbox2segment("/content/VMdataset")

Scanning /content/VMdataset/labels.cache... 497 images, 1 backgrounds, 0 corrupt: 100%|██████████| 497/497 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
Detection labels detected, generating segment labels by SAM model!



Generating segment labels: 100%|██████████| 497/497 [03:27<00:00,  2.39it/s]


Generated segment labels saved in /content/labels-segment


Конвертация класса 0 в сегментированные аннотации заняла 4 минуты


In [ ]:
!cat /content/labels-segment/00001.txt

0 0.248437 0.55 0.246875 0.551562 0.242188 0.551562 0.239063 0.554688 0.235938 0.554688 0.235938 0.55625 0.234375 0.557813 0.234375 0.560938 0.232812 0.5625 0.232812 0.565625 0.23125 0.567187 0.23125 0.578125 0.229687 0.579687 0.229687 0.58125 0.228125 0.582812 0.226562 0.582812 0.225 0.584375 0.223438 0.584375 0.223438 0.585938 0.221875 0.5875 0.221875 0.589063 0.220313 0.590625 0.220313 0.6 0.221875 0.601562 0.221875 0.607813 0.223438 0.609375 0.223438 0.610937 0.225 0.609375 0.225 0.60625 0.226562 0.604688 0.226562 0.603125 0.228125 0.601562 0.23125 0.601562 0.232812 0.603125 0.232812 0.607813 0.23125 0.609375 0.232812 0.610937 0.232812 0.614062 0.2375 0.61875 0.2375 0.620313 0.239063 0.621875 0.239063 0.626562 0.242188 0.626562 0.245312 0.629687 0.245312 0.6375 0.24375 0.639063 0.240625 0.639063 0.239063 0.640625 0.239063 0.664062 0.242188 0.667188 0.242188 0.66875 0.24375 0.670313 0.251563 0.670313 0.253125 0.671875 0.265625 0.671875 0.267188 0.673437 0.275 0.673437 0.276563 0.675

In [ ]:
!cat /content/labels-segment/00497.txt

0 0.0640625 0.315625 0.0625 0.317187 0.0609375 0.317187 0.0609375 0.31875 0.059375 0.320312 0.059375 0.3375 0.0609375 0.339063 0.0609375 0.340625 0.0625 0.342187 0.0625 0.34375 0.0640625 0.345313 0.0640625 0.348437 0.065625 0.35 0.065625 0.351562 0.0671875 0.353125 0.0671875 0.35625 0.0703125 0.359375 0.0703125 0.3625 0.071875 0.364062 0.071875 0.367188 0.0734375 0.36875 0.0734375 0.370313 0.075 0.371875 0.075 0.373437 0.078125 0.376563 0.078125 0.379687 0.08125 0.382812 0.08125 0.384375 0.0828125 0.385938 0.0828125 0.3875 0.0859375 0.390625 0.0859375 0.392188 0.0875 0.39375 0.0875 0.395312 0.0921875 0.4 0.0921875 0.401563 0.09375 0.403125 0.09375 0.404687 0.0953125 0.40625 0.0953125 0.407813 0.096875 0.409375 0.096875 0.410937 0.0984375 0.4125 0.0984375 0.414062 0.1 0.415625 0.1 0.417188 0.101562 0.41875 0.101562 0.420312 0.103125 0.421875 0.103125 0.426562 0.10625 0.429688 0.10625 0.432813 0.107813 0.434375 0.107813 0.435937 0.109375 0.4375 0.109375 0.44375 0.110937 0.445312 0.110937

In [ ]:
import shutil

# Удаление всех файлов и поддиректорий из указанной директории
shutil.rmtree('/content/drive/MyDrive/labels_obb_new')



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

**Преобразование полигональной разметки в разметку OBB**

In [ ]:
# ---------


# Проверка координат прямоугольника
def  check_coords( rect ):
  for i in range(4):
    for j in range(2):
      if rect[i, j] > 1:
        return False
  return True

# Преобразование координат полигона в обычный ограничивающий прямоугольник
def convert_to_rect(points):
  # Преобразуем список точек в массив numpy
  points = np.array(points, dtype=np.float32)

  # Извлекаем x и y координаты
  x_coords = points[::2]
  y_coords = points[1::2]

  min_x = np.min(x_coords)
  max_x = np.max(x_coords)
  min_y = np.min(y_coords)
  max_y = np.max(y_coords)

  rect_coords = np.array([[ max_x, min_y], [max_x, max_y], [min_x, max_y], [min_x, min_y]])

  return rect_coords



# Преобразование координат полигона в  ограничивающий прямоугольник минимальной площади
def convert_to_rotated_rect(points):
    # Преобразуем список точек в массив numpy
    points = np.array(points, dtype=np.float32)
    x_coords = points[::2]
    y_coords = points[1::2]
    pairs = np.column_stack((x_coords, y_coords))

    # Нахождение ограничивающего прямоугольника минимальной площади
    rect = cv2.minAreaRect(pairs)
    # Преобразуем его в точки
    box = cv2.boxPoints(rect)
    box = np.float32(box)
    return box

# Считывание данных из файла с сегментацией,
# преобразование в OBB ограничивающие прямоугольники минимальной площади
# запись координат прямоугольника в файл аннотации
def segment_to_obb_( segment_filename, obblabel_filename ):
  # Считываем данные из файла с сегментацией в формате YOLO
  with open(segment_filename, 'r') as file:
    yolo_data = file.readlines()

  # Преобразуем данные в ориентированные прямоугольники
  rotated_rect_data = []
  for line in yolo_data:
    class_id, *points = line.split()
    rotated_rect = convert_to_rotated_rect(points)
    # print('rotated_data ',rotated_rect)
    if check_coords( rotated_rect ):
      rotated_rect_data.append((class_id, rotated_rect))
    else:
      bbox = convert_to_rect( points)
      print("Координаты ограничивающего прямоугольника:", bbox[0,0], bbox[0,1], bbox[1,0], bbox[1,1], bbox[2,0], bbox[2,1], bbox[3,0], bbox[3,1])
      rotated_rect_data.append((class_id, bbox))

  # Записываем данные в новый файл разметки
  with open(obblabel_filename, 'w') as file:
    for class_id, rect_coords in rotated_rect_data:
        file.write(f"{class_id} {rect_coords[0,0]} {rect_coords[0,1]} {rect_coords[1,0]} {rect_coords[1,1]}  {rect_coords[2,0]}  {rect_coords[2,1]} {rect_coords[3,0]}  {rect_coords[3,1]}\n")
#-----------------------------------------------------------

In [ ]:
import os

# Путь к директориям с  аннотациями контуров сегментации - исходные данные
path_to_segments = '/content/labels-segment'
# Путь к директориям с  аннотациями OBB - целевые данные
path_to_labels = '/content/drive/MyDrive/labels_obb_new'

if os.path.exists(path_to_labels)==False:
  os.mkdir(path_to_labels)
# удаление старых файлов аннотаций
for filename in os.listdir(path_to_labels):
    filepath = os.path.join(path_to_labels, filename)
    if os.path.isfile(filepath):
      os.remove(filepath)




# Получаем список файлов сегментаций
segment_files = os.listdir(path_to_segments)
segment_files.sort()

for segm_file in segment_files:
    segment_path = os.path.join(path_to_segments, segm_file)
    label_filename = os.path.splitext(segm_file)[0] + '.txt'
    label_path = os.path.join(path_to_labels, label_filename)

    segment_to_obb_(segment_path, label_path)

Координаты ограничивающего прямоугольника: 0.576563 0.703125 0.576563 0.998438 0.3875 0.998438 0.3875 0.703125
Координаты ограничивающего прямоугольника: 0.928125 0.0203125 0.928125 0.995313 0.071875 0.995313 0.071875 0.0203125
Координаты ограничивающего прямоугольника: 0.990625 0.135937 0.990625 0.875 0.0421875 0.875 0.0421875 0.135937
Координаты ограничивающего прямоугольника: 0.970312 0.0828125 0.970312 0.939062 0.0171875 0.939062 0.0171875 0.0828125
Координаты ограничивающего прямоугольника: 0.892187 0.04375 0.892187 0.9625 0.003125 0.9625 0.003125 0.04375
Координаты ограничивающего прямоугольника: 0.998438 0.0875 0.998438 0.917188 0.04375 0.917188 0.04375 0.0875
Координаты ограничивающего прямоугольника: 0.979688 0.135937 0.979688 0.879687 0.0140625 0.879687 0.0140625 0.135937
Координаты ограничивающего прямоугольника: 0.998438 0.08125 0.998438 0.921875 0.0 0.921875 0.0 0.08125
Координаты ограничивающего прямоугольника: 0.998438 0.1375 0.998438 0.892187 0.003125 0.892187 0.003125 

In [ ]:
!cat /content/labels_obb_new/00001.txt

0 0.20790936052799225 0.6760721206665039 0.22741414606571198 0.541705846786499  0.5362535715103149  0.5865373611450195 0.5167487859725952  0.7209036350250244
0 0.41910862922668457 0.35570028424263 0.4803299307823181 0.10570642352104187  0.6157660484313965  0.13887354731559753 0.5545447468757629  0.38886740803718567
0 0.6506938338279724 0.24739496409893036 0.8113183379173279 0.16107313334941864  0.9628871083259583  0.44310665130615234 0.8022626042366028  0.5294284820556641
0 0.534738302230835 0.5147243738174438 0.7597707509994507 0.4449470043182373  0.881854772567749  0.838668942451477 0.6568223237991333  0.9084463119506836
0 0.18437501788139343 0.7624999284744263 0.4296880066394806 0.7624999284744263  0.4296880066394806  0.9984380006790161 0.18437501788139343  0.9984380006790161


In [ ]:
!cat /content/drive/MyDrive/labels_obb_new/00497.txt

0 0.05485982447862625 0.31772640347480774 0.2183225005865097 0.2804006040096283  0.2878926694393158  0.5850726366043091 0.12442998588085175  0.6223983764648438
0 0.24317499995231628 0.11231791973114014 0.31786683201789856 0.09561660885810852  0.37647709250450134  0.3577340245246887 0.30178526043891907  0.37443533539772034
0 0.171875 0.8046879768371582 0.171875 0.9984380006790161  0.0031250000465661287  0.9984380006790161 0.0031250000465661287  0.8046879768371582
0 0.1588841676712036 0.8562755584716797 0.1856120228767395 0.793414831161499  0.3944801688194275  0.8822236061096191 0.3677523136138916  0.9450843334197998
0 0.4285019636154175 0.3772316873073578 0.6361279487609863 0.34497901797294617  0.696579098701477  0.7341321706771851 0.4889531135559082  0.7663848400115967
0 0.29648494720458984 0.8958439826965332 0.3637096881866455 0.5915647745132446  0.48249387741088867  0.6178078651428223 0.415269136428833  0.9220870733261108
0 0.6410311460494995 0.48153120279312134 0.6777812242507935 0.

In [ ]:
'''Dataset
   ├─images
     ├─train
      ├─ ..jpg
     ├─val
      ├─ ..jpg
  ├─labels
    ├─train
      ├─ ..txt
    ├─val
      ├─ ..txt
  data.yaml
  '''

In [ ]:
import shutil

# Удаление всех файлов и поддиректорий из указанной директории
shutil.rmtree('/content/drive/MyDrive/OBBdataset')


In [ ]:

import os
import random
import shutil

def remove_file( file_path ):
  # Проверяем существование файла перед удалением
  if os.path.exists(file_path):
    os.remove(file_path)

# функция преобразования директории изображений и директории аннотаций в директорию датасета для обучения YOLO
def  prepare_yolo_dataset( scr_images_dir, scr_labels_dir, dataset_path ):
  # Пути к целевым директориям
  train_images_dir = f'{dataset_path}/images/train'
  val_images_dir = f'{dataset_path}/images/val'
  train_labels_dir = f'{dataset_path}/labels/train'
  val_labels_dir = f'{dataset_path}/labels/val'
  # print('train_images_dir=', train_images_dir)
  # print('train_labels_dir=', train_labels_dir)

  # Создание целевых директорий, если они не существуют
  os.makedirs(train_images_dir, exist_ok=True)
  os.makedirs(val_images_dir, exist_ok=True)
  os.makedirs(train_labels_dir, exist_ok=True)
  os.makedirs(val_labels_dir, exist_ok=True)

  # Получение списка файлов *.jpg и *.txt
  image_files = [f for f in os.listdir(scr_images_dir) if f.endswith('.jpg')]
  label_files = [f for f in os.listdir(scr_labels_dir) if f.endswith('.txt')]
  image_files.sort()
  label_files.sort()

  # Перемешивание файлов
 # random.shuffle(image_files)
 # random.shuffle(label_files)

  # Разделение на train и val
  split_idx = int(0.8 * len(image_files))

  train_images = image_files[:split_idx]
  val_images = image_files[split_idx:]

  train_labels = label_files[:split_idx]
  val_labels = label_files[split_idx:]

  # Копирование файлов в целевые директории
  for img in train_images:
    shutil.copy(os.path.join(scr_images_dir, img), os.path.join(train_images_dir, img))
  for img in val_images:
    shutil.copy(os.path.join(scr_images_dir, img), os.path.join(val_images_dir, img))

  for lbl in train_labels:
    shutil.copy(os.path.join(scr_labels_dir, lbl), os.path.join(train_labels_dir, lbl))
  for lbl in val_labels:
    shutil.copy(os.path.join(scr_labels_dir, lbl), os.path.join(val_labels_dir, lbl))

  # Создание файла data.yaml
  with open(dataset_path + '/data.yaml', 'w') as f:
    f.write("train: images/train\n")
    f.write("val: images/val\n")
    f.write("nc: 4\n")
    f.write("names: ['enjine', 'container','ballon', 'radiator']")



In [ ]:
# ПОДГОТОВКА ДАТАСЕТА ДЛЯ ОБУЧЕНИЯ МОДЕЛИ
prepare_yolo_dataset( '/content/VMdataset/images', '/content/drive/MyDrive/labels_obb_new', '/content/drive/MyDrive/OBBdataset' )

In [ ]:
!cat /content/drive/MyDrive/OBBdataset/labels/val/00497.txt

0 0.05485982447862625 0.31772640347480774 0.2183225005865097 0.2804006040096283  0.2878926694393158  0.5850726366043091 0.12442998588085175  0.6223983764648438
0 0.24317499995231628 0.11231791973114014 0.31786683201789856 0.09561660885810852  0.37647709250450134  0.3577340245246887 0.30178526043891907  0.37443533539772034
0 -0.0034780502319335938 0.9615892767906189 0.15334060788154602 0.7990321516990662  0.23676685988903046  0.8795133233070374 0.07994820177555084  1.0420703887939453
0 0.1588841676712036 0.8562755584716797 0.1856120228767395 0.793414831161499  0.3944801688194275  0.8822236061096191 0.3677523136138916  0.9450843334197998
0 0.4285019636154175 0.3772316873073578 0.6361279487609863 0.34497901797294617  0.696579098701477  0.7341321706771851 0.4889531135559082  0.7663848400115967
0 0.29648494720458984 0.8958439826965332 0.3637096881866455 0.5915647745132446  0.48249387741088867  0.6178078651428223 0.415269136428833  0.9220870733261108
0 0.6410311460494995 0.48153120279312134 

**ОБУЧЕНИЕ МОДЕЛИ YOLOv8**

In [ ]:
from ultralytics import YOLO

# Load a pretrained model
model = YOLO("yolov8l-obb.pt")

# Train the model
results = model.train(data="/content/drive/MyDrive/OBBdataset/data.yaml", epochs=20, imgsz=640)

100%|██████████| 85.4M/85.4M [00:00<00:00, 134MB/s]


Ultralytics YOLOv8.2.74 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=obb, mode=train, model=yolov8l-obb.pt, data=/content/drive/MyDrive/OBBdataset/data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sho

100%|██████████| 755k/755k [00:00<00:00, 24.0MB/s]


Overriding model.yaml nc=15 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  3    279808  ultralytics.nn.modules.block.C2f             [128, 128, 3, True]           
  3                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  4                  -1  6   2101248  ultralytics.nn.modules.block.C2f             [256, 256, 6, True]           
  5                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  6                  -1  6   8396800  ultralytics.nn.modules.block.C2f             [512, 512, 6, True]           
  7                  -1  1   2360320  ultralytics

100%|██████████| 6.25M/6.25M [00:00<00:00, 138MB/s]


...

      20/20      11.4G     0.7962      0.756      2.355          7        640: 100%|██████████| 25/25 [00:19<00:00,  1.27it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:03<00:00,  2.26it/s]

                   all        100        160      0.744      0.617      0.668      0.347



20 epochs completed in 0.151 hours.
Optimizer stripped from runs/obb/train/weights/last.pt, 89.4MB
Optimizer stripped from runs/obb/train/weights/best.pt, 89.4MB

Validating runs/obb/train/weights/best.pt...
Ultralytics YOLOv8.2.74 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8l-obb summary (fused): 287 layers, 44,458,143 parameters, 0 gradients, 168.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:05<00:00,  1.27it/s]


                   all        100        160      0.706      0.625      0.666      0.352
                enjine         99        160      0.706      0.625      0.666      0.352
Speed: 0.5ms preprocess, 17.7ms inference, 0.0ms loss, 12.0ms postprocess per image
Results saved to runs/obb/train


Время обучения 9 минут
